<a href="https://colab.research.google.com/github/polyphemus287/archaeological_site_wikidata/blob/main/archeological_site_wikidata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install SPARQLWrapper
!pip install requests folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.2 MB/s eta 0:00:00


In [ ]:
from IPython.display import display, HTML
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import folium
import time  # Import the time module
import os
import webbrowser
import urllib.parse

In [ ]:
# SPARQL query
query = """
SELECT ?site ?siteLabel ?coordinates ?discoveryDate ?description ?countryLabel ?image WHERE {
  ?site wdt:P31 wd:Q839954.
  ?site wdt:P625 ?coordinates.
  OPTIONAL { ?site wdt:P571 ?discoveryDate. }
  OPTIONAL { ?site schema:description ?description FILTER(LANG(?description) = "en"). }
  OPTIONAL { ?site wdt:P18 ?image. }
  OPTIONAL { ?site wdt:P131* ?country. ?country wdt:P31 wd:Q6256. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
LIMIT 750
"""

# Endpoint URL
url = 'https://query.wikidata.org/sparql'

# Send request
r = requests.get(url, params={'format': 'json', 'query': query})

# Parse results
data = r.json()

# Create map
map_ = folium.Map(location=[20, 0], zoom_start=2)

# Initialize a set to track plotted coordinates
plotted_coordinates = set()

for item in data['results']['bindings']:
    # Extract coordinates and ensure correct order
    coords = item['coordinates']['value'][6:-1].split(' ')
    lon, lat = map(float, coords)  # Ensure longitude and latitude are correctly ordered
    lat_rounded = round(lat, 5)  # Round latitude for consistency
    lon_rounded = round(lon, 5)  # Round longitude for consistency

    # Create a string representation of the rounded coordinates for tracking
    coord_str = f"{lat_rounded},{lon_rounded}"
    if coord_str in plotted_coordinates:
        continue  # Skip the plotting for this item if its coordinates have already been plotted

    # Extract label and other information
    name = item['siteLabel']['value']
    description = item.get('description', {}).get('value', 'No description available')
    discoveryDate = item.get('discoveryDate', {}).get('value', 'Unknown discovery date')
    image = item.get('image', {}).get('value', '')

    # Construct image HTML if available
    if image:
        filename = urllib.parse.unquote(image.split('/')[-1])
        filename = urllib.parse.quote(filename)
        image_page_url = f"https://commons.wikimedia.org/wiki/File:{filename}"
        image_html_direct_url = f"https://commons.wikimedia.org/wiki/Special:FilePath/{filename}?width=300"  # Correctly encoded direct link to the image for embedding
        image_html = f'<img src="{image_html_direct_url}" style="width:100%;max-height:200px;margin-top:10px;"><br>'
        link_html = f'<a href="{image_page_url}" target="_blank">View Full Image</a>'
    else:
        image_html = 'No image available<br>'
        link_html = ''

    popup_html = f"""
        <div>
            <strong>{name}</strong><br>
            {description}<br>
            Discovery Date: {discoveryDate}<br>
            {image_html}
            {link_html}
        </div>
    """
    popup = folium.Popup(html=popup_html, max_width=300)

    # Print the location and coordinates being plotted for verification
    print(f"Plotting: {name} at Latitude: {lat_rounded}, Longitude: {lon_rounded}")

    # Add marker to the map
    folium.Marker([lat_rounded, lon_rounded], popup=popup).add_to(map_)

    # Add the rounded coordinates to the set to track what's been plotted
    plotted_coordinates.add(coord_str)

    # Introduce a brief pause (optional, for demonstration)
    time.sleep(0.15)

# Display the map inline
display(map_)


Plotting: Tassili n'Ajjer at Latitude: 25.66667, Longitude: 9.0
Plotting: Leptis Magna at Latitude: 32.63833, Longitude: 14.2905
Plotting: Heliopolis at Latitude: 30.12953, Longitude: 31.28889
Plotting: Saqqara at Latitude: 29.85, Longitude: 31.23333
Plotting: Abydos at Latitude: 26.18517, Longitude: 31.91889
Plotting: Pyramid of Djoser at Latitude: 29.87127, Longitude: 31.21639
Plotting: Çatalhöyük at Latitude: 37.6675, Longitude: 32.82833
Plotting: Ctesiphon at Latitude: 33.09361, Longitude: 44.58083
Plotting: Trajan's Column at Latitude: 41.89583, Longitude: 12.48417
Plotting: Minaret of Jam at Latitude: 34.39639, Longitude: 64.51589
Plotting: Platonic Academy at Latitude: 37.99236, Longitude: 23.70806
Plotting: Merv at Latitude: 37.66278, Longitude: 62.1925
Plotting: Thermopylae at Latitude: 38.79762, Longitude: 22.53579
Plotting: Atarneus at Latitude: 39.09136, Longitude: 26.92131
Plotting: Abu Salabikh at Latitude: 32.26667, Longitude: 45.08333
Plotting: Tell al-Rimah at Latitude